In [1]:
from caveclient import CAVEclient
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from standard_transform import minnie_ds
import numpy as np
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

datastack_name = 'minnie65_public'
client = CAVEclient(datastack_name)

# Show the description of the datastack
client.info.get_datastack_info()['description']

# see the available materialization versions
client.materialize.get_versions()

versions = sorted(client.materialize.get_versions(), key = lambda v: client.materialize.get_timestamp(v))

for version in versions:
    print(f"Version {version}: {client.materialize.get_timestamp(version)}")

# CAVE always use the same version for consistency
client.version = 1621

Version 117: 2021-06-11 08:10:00.215114+00:00
Version 343: 2022-02-24 08:10:00.184668+00:00
Version 661: 2023-04-06 20:17:09.199182+00:00
Version 795: 2023-08-23 08:10:01.404268+00:00
Version 943: 2024-01-22 08:10:01.497934+00:00
Version 1078: 2024-06-05 10:10:01.203215+00:00
Version 1181: 2024-09-16 10:10:01.121167+00:00
Version 1300: 2025-01-13 10:10:01.286229+00:00
Version 1412: 2025-04-29 10:10:01.200893+00:00
Version 1507: 2025-07-31 08:10:01.117494+00:00
Version 1621: 2025-11-25 08:10:01.094430+00:00


In [2]:
client.materialize.get_tables()

['baylor_gnn_cell_type_fine_model_v2',
 'vortex_manual_myelination_v0',
 'synapse_target_predictions_ssa',
 'aibs_metamodel_celltypes_v661',
 'nucleus_alternative_points',
 'allen_column_mtypes_v2',
 'bodor_pt_cells',
 'aibs_metamodel_mtypes_v661_v2',
 'vortex_microglia_proofreading_status',
 'proofreading_status_and_strategy',
 'allen_v1_column_types_slanted_ref',
 'aibs_column_nonneuronal_ref',
 'nucleus_ref_neuron_svm',
 'apl_functional_coreg_vess_fwd',
 'vortex_axon_backtrace_column',
 'vortex_compartment_targets',
 'baylor_log_reg_cell_type_coarse_v1',
 'vortex_synapse_reattachment',
 'gamlin_2023_mcs_met_types',
 'gamlin_2023_mcs',
 'l5et_column',
 'pt_synapse_targets',
 'coregistration_manual_v4',
 'cg_cell_type_calls',
 'synapses_pni_2',
 'nucleus_detection_v0',
 'vortex_manual_nodes_of_ranvier',
 'bodor_pt_target_proofread',
 'nucleus_functional_area_assignment',
 'coregistration_auto_phase3_fwd_apl_vess_combined_v2',
 'vortex_thalamic_proofreading_status',
 'multi_input_spine

In [3]:
client.materialize.get_table_metadata('proofreading_status_and_strategy')

{'table_name': 'proofreading_status_and_strategy',
 'aligned_volume': 'minnie65_phase3',
 'id': 80389,
 'schema': 'compartment_proofread_status_strategy',
 'valid': True,
 'created': '2024-06-03T19:46:04.824405',
 'schema_type': 'compartment_proofread_status_strategy',
 'user_id': '4447',
 'description': 'The proofreading status of neurons that have been manually cleaned, extended, or both. Axon and dendrite compartment status are marked separately under status_axon and status_dendrite, as proofreading effort was applied differently to the different compartments in some cells. status_axon and status_dendrite are TRUE if the compartment is at least clean, meaning the synapses are accurate but possibly incomplete. strategy_axon and strategy_dendrite represent the specific strategy used for each compartment, the full details of which are available at www.microns-explorer.org/manifests/mm3-proofreading. Uploaded and maintained by Bethanny Danskin, with the help of Forrest Collman and and C

In [4]:
proofread_df = client.materialize.tables.proofreading_status_and_strategy().query(split_positions = True, limit = 1000000)
proofread_df

,id,created,superceded_id,valid,pt_position_x,pt_position_y,pt_position_z,valid_id,status_dendrite,status_axon,strategy_dendrite,strategy_axon,pt_supervoxel_id,pt_root_id
0,1526,2024-08-02 20:25:56.067777+00:00,<NA>,True,169536,120704,21696,864691136195824204,True,True,dendrite_extended,axon_partially_extended,88109708761918664,864691136195824204
1,1533,2024-08-02 20:25:56.173122+00:00,<NA>,True,185936,133408,20157,864691136674274951,True,True,dendrite_extended,axon_partially_extended,90363226361177973,864691136674274951
2,1535,2024-08-02 20:25:56.201735+00:00,<NA>,True,172352,139920,21312,864691136330407914,True,True,dendrite_clean,axon_partially_extended,88464095036445073,864691136330407914
3,1538,2024-08-02 20:25:56.242728+00:00,<NA>,True,187888,124512,21300,864691136904595890,True,True,dendrite_clean,axon_partially_extended,90573095777456657,864691136904595890
4,3625,2025-04-24 22:35:11.897285+00:00,<NA>,True,184160,183312,22166,864691135280481697,True,True,dendrite_extended,axon_partially_extended,90088417442091280,864691135280481697
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2309,3996,2025-07-22 18:26:32.692898+00:00,<NA>,True,167472,115920,22692,864691136390675216,True,True,dendrite_extended,axon_fully_extended,87827615444091119,864691136390675216
2310,3909,2025-07-22 18:26:31.122404+00:00,<NA>,True,190960,125888,22490,864691135875426190,True,True,dendrite_clean,axon_partially_extended,90995514535370826,864691136331104234
2311,3998,2025-07-22 18:26:32.732018+00:00,<NA>,True,190080,199008,23676,864691136312076122,True,True,dendrite_extended,axon_fully_extended,90934972877275944,864691136312076122
2312,4000,2025-07-22 18:26:32.763822+00:00,<NA>,True,206288,194112,20873,864691135464595518,True,True,dendrite_extended,axon_fully_extended,93115716349687539,864691135464595518


In [ ]:
cell_type_df = client.materialize.tables.aibs_metamodel_celltypes_v661().query(split_positions = True, select_columns = ['pt_position', 'pt_root_id', 'cell_type'], limit = 1000000)
cell_type_df

In [ ]:
# what cells are proofread?

proofread_list = proofread_df['valid_id'].tolist()

proofread_cell_types = cell_type_df[cell_type_df['pt_root_id'].isin(proofread_list)].copy().reset_index(drop = True)
print(proofread_cell_types)

In [ ]:
from functions import analysis
analysis(proofread_cell_types)

In [ ]:
ct_df = client.materialize.query_table('proofreading_status_and_strategy', split_positions=True)


X_transformed = minnie_ds.transform_vx.apply_dataframe('pt_position', ct_df)
X_transformed = np.array(X_transformed)
ct_df['pt_xt'] = X_transformed[:,0]
ct_df['pt_yt'] = X_transformed[:,1]
ct_df['pt_zt'] = X_transformed[:,2] 

fig, ax = plt.subplots(figsize=(5,4))

sns.scatterplot(x = 'pt_xt', y = 'pt_yt', s = 3, data = ct_df.sample(2_000), ax = ax)

ax.invert_yaxis()
sns.despine(ax = ax)

In [ ]:
X_transformed = minnie_ds.transform_vx.apply_dataframe('pt_position', ct_df)
X_transformed = np.array(X_transformed)
ct_df['pt_xt'] = X_transformed[:,0]
ct_df['pt_yt'] = X_transformed[:,1]
ct_df['pt_zt'] = X_transformed[:,2] 

fig, ax = plt.subplots(figsize = (5,4))

sns.scatterplot(x = 'pt_zt', y = 'pt_yt', s = 3, data = ct_df.sample(2_000), ax = ax)

ax.invert_yaxis()
sns.despine(ax = ax)